In [1]:
# # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !pip install transformers
# !pip install transformers scipy ftfy accelerate

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [1]:
from recomendaciones_comsint import Recomendador

c:\Users\jsaur\anaconda3\envs\ENV1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
recomendador128 = Recomendador(fuente='recipes_con_info_nutricional.csv')
recomendador128.CargarModelo(emb_size=128, version=4)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Modelo Modelos/Modelo_Nut_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.


In [10]:
recomendador128.FiltrarRecetario_por_CanastaBasica(similitud=0.5, verbose=False)
df_platillos_recomendados2 = recomendador128.Calcular_InfoNutricional(verbose=False)

Buscando recetas con ingredientes de la canasta básica... 



100%|██████████| 1090/1090 [00:27<00:00, 39.37it/s]


 

 20 platillos encontrados con similitud mayor a 0.5


In [ ]:
df2 = df_platillos_recomendados2.sort_values(by=['similitud'], ascending=False)
df2[df2['puntaje_platillo']>0.6]

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
2,Receta de Mole de olla de pollo,1 kilogramo de Piezas de pollo\n 2 piezas de E...,0.6,473.94,20.64,60.42,12.25,0.7
7,Receta de Aporreadillo rojo,500 gramos de Cesina\n 500 gramos de Jitomate\...,0.6,575.91,26.84,70.50,17.29,0.7
8,Receta de Discada norteña,00 gramos de Carne de res\n 500 gramos de Pulp...,0.6,1411.85,76.03,123.44,69.23,0.7
9,Guiso de filete a la mexicana,3 unidades de Jitomates maduros picados en cub...,0.6,959.68,61.38,69.97,50.30,0.7
14,Receta de Chayotes a la mexicana,1 chayote\n 1 jitomate\n 1 cebolla chica\n 2 ...,0.6,446.81,23.91,51.57,13.35,0.7
0,Receta de Carne con chile colorado,1 kilogramo de carne de res para deshebrar\n 4...,0.5,1630.01,83.11,122.29,92.50,0.7
10,Receta de Estofado de res ranchero,800 gramos de Carne (paceto o colita de cuadri...,0.5,727.75,37.14,56.70,37.93,0.7
12,Receta de Pozole michoacano,1 lata grande de maiz pozolero\n 2kg de jitoma...,0.4,773.70,36.13,67.07,37.74,0.7


In [10]:
ingredientes = ["4 huevos, 1 taza de azúcar mascabado, 200 gramos de harina de trigo común, 1 limón (su piel), 1 litro de leche semidesnatada, 2 cucharadas soperas de agua de azahar o anís, 3 cucharadas soperas de miel, 2 vasos de aceite de girasol, 2 cucharadas soperas de anís en grano (anisetes), 1 cucharada sopera de canela en polvo",
"400 gr carne molida de res 80% carne 20%grasa o algo parecido, 1 cucharada aderezo ranch,4 rebanadas de tocino,Queso Monterrey Jack al gusto,2 panes de hamburguesa,½ tomate, 4 hojas de lechuga, 1/8 cebolla blanca fileteada, Pepinillos opcional, Mayonesa mostaza, cátsup al gusto, Sal y pimienta al gusto"
]
 
recomendador128.Calcular_InfoNutricional_from_List(ingredientes, verbose=False)

1/1 [==============================] - 0s 33ms/step


,ingredientes,kcal,proteinas_gr,carbs_gr,grasas_gr,puntaje_platillo
0,"4 huevos, 1 taza de azúcar mascabado, 200 gram...",264.01,6.88,32.72,11.12,0.7
1,400 gr carne molida de res 80% carne 20%grasa ...,252.38,14.05,9.87,16.85,0.7
